In [3]:
train_data.columns

Index(['label', 'title_sum', 'title_count', 'title_ctr',
       'title_query_dot_similarity', 'title_query_wmd_dis', 'query_word_num',
       'title_word_num', 'is_query_in_title', 'query_len', 'title_len',
       'len_title-query', 'len_query/title', 'q_t_word_match_share',
       'q_t_jaccard', 'q_t_common_words', 'q_t_total_unique_words',
       'q_t_wc_diff', 'q_t_wc_ratio', 'q_t_wc_diff_unique',
       'q_t_wc_ratio_unique', 'title_nunique_prefix', 'Levenshtein_dis',
       'Levenshtein_ratio', 'Levenshtein_jaro', 'Levenshtein_jaro_winkler',
       'longistStr_len', 'prefix_title_pos', 'prefix_title_relative_pos',
       'len_title-query_word', 'len_query/title_word',
       'title_query_leven_rate', 'lcseque_len', 'preifx_detail_title_pos',
       'cityblock_sim', 'norm_similarity', 'cosine_similarity', '0_lsi',
       '1_lsi', '2_lsi', '3_lsi', '4_lsi', '5_lsi', '6_lsi', '7_lsi', '8_lsi',
       '9_lsi', '10_lsi', '11_lsi', '12_lsi', '13_lsi', '14_lsi', '15_lsi',
       '16_lsi'

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import time
import sys
import datetime
import gc
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectPercentile
import math
from sklearn.metrics import f1_score
import jieba
import jieba.posseg as psg
from collections import Counter
import functools
from time import time
from sklearn import preprocessing

import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
from sklearn import metrics
import os, time, datetime
import numpy as np
from sklearn import preprocessing


def norm(train_df,features):    
    scaler = preprocessing.QuantileTransformer(random_state=0) 
    scaler.fit(train_df[features])  
    train_df[features]=scaler.transform(train_df[features]) 

train_data=pd.read_pickle("/home/kesci/train_data9-9.5.pickle")
print(train_data.shape)

# norm(train_data,['title_count','title_sum','title_nunique_prefix'])

# test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_feature_first_no_query_and_title_data.pickle")
fea=list(train_data.columns)
# print(fea)
fea.remove('label')
# fea.remove('query_title_types')

drop_list=[ ]
for i in drop_list:########删除一些不参与训练的特征
    fea.remove(i)
    
print(fea)
import numpy as np
for i in train_data:#########因为当时5千万数据，所有特征没办法直接加载到内存，需要降低内存消耗
    if isinstance(train_data[i],np.float64):
        train_data[i]=train_data[i].astype(np.float32)
    if isinstance(train_data[i],np.int64):
        train_data[i]=train_data[i].astype(np.int16)


#############此处划分训练集的方式，建议改成 DataFrame.sample()
data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.002,random_state=666)
train_index,vaild_index=data_split.split(train_data['label'],train_data['label']).__next__()
lgb_train = lgb.Dataset(train_data[fea], train_data['label'])
lgb_eval = lgb.Dataset(train_data[fea].iloc[vaild_index], train_data['label'].iloc[vaild_index], reference=lgb_train)

del train_data
import gc
gc.collect()
params = {
        'boosting_type':'gbdt', 'num_leaves':63, 'max_depth':-1, 'n_estimators':1200, 'objective':'binary',
        'subsample':0.8, 'colsample_bytree':0.8, 'subsample_freq':1,'num_boost_round':1000,
        'learning_rate':0.007, 'random_state':666, 'silent':False,'verbose':1,
        'reg_alpha':0.0,'reg_lambda':1,'feature_fraction':0.8,'min_child_weight':50
}
# print("start2")
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                verbose_eval=2,
                #  init_model='/home/kesci/work/first_zzp/model/no_title_newn_norm.txt'
                )

(50000000, 57)
['title_sum', 'title_count', 'title_ctr', 'title_query_dot_similarity', 'title_query_wmd_dis', 'query_word_num', 'title_word_num', 'is_query_in_title', 'query_len', 'title_len', 'len_title-query', 'len_query/title', 'q_t_word_match_share', 'q_t_jaccard', 'q_t_common_words', 'q_t_total_unique_words', 'q_t_wc_diff', 'q_t_wc_ratio', 'q_t_wc_diff_unique', 'q_t_wc_ratio_unique', 'title_nunique_prefix', 'Levenshtein_dis', 'Levenshtein_ratio', 'Levenshtein_jaro', 'Levenshtein_jaro_winkler', 'longistStr_len', 'prefix_title_pos', 'prefix_title_relative_pos', 'len_title-query_word', 'len_query/title_word', 'title_query_leven_rate', 'lcseque_len', 'preifx_detail_title_pos', 'cityblock_sim', 'norm_similarity', 'cosine_similarity', '0_lsi', '1_lsi', '2_lsi', '3_lsi', '4_lsi', '5_lsi', '6_lsi', '7_lsi', '8_lsi', '9_lsi', '10_lsi', '11_lsi', '12_lsi', '13_lsi', '14_lsi', '15_lsi', '16_lsi', '17_lsi', '18_lsi', '19_lsi']


/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:147: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:755: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[2]	valid_0's binary_logloss: 0.474023
[4]	valid_0's binary_logloss: 0.473229
[6]	valid_0's binary_logloss: 0.472439
[8]	valid_0's binary_logloss: 0.471922
[10]	valid_0's binary_logloss: 0.471173
[12]	valid_0's binary_logloss: 0.470452
[14]	valid_0's binary_logloss: 0.469755
[16]	valid_0's binary_logloss: 0.469073
[18]	valid_0's binary_logloss: 0.46848
[20]	valid_0's binary_logloss: 0.468112
[22]	valid_0's binary_logloss: 0.467884
[24]	valid_0's binary_logloss: 0.467255
[26]	valid_0's binary_logloss: 0.466705
[28]	valid_0's binary_logloss: 0.466181
[30]	valid_0's binary_logloss: 0.465653
[32]	valid_0's binary_logloss: 0.465086
[34]	valid_0's binary_logloss: 0.464536
[36]	valid_0's binary_logloss: 0.464059
[38]	valid_0's binary_logloss: 0.463591
[40]	valid_0's binary_logloss: 0.463088
[42]	valid_0's binary_logloss: 0.462601
[44]	valid_0's binary_logloss: 0.462171
[46]	valid_0's binary_logloss: 0.461703
[48]	valid_0's binary_logloss: 0.461291
[50]	valid_0's binary_logloss: 0.460851
[52]	

In [2]:
for index,i in enumerate(fea):
    print(i,":",gbm.feature_importance()[index])

title_sum : 2849
title_count : 2281
title_ctr : 5588
title_query_dot_similarity : 2612
title_query_wmd_dis : 862
query_word_num : 1470
title_word_num : 298
is_query_in_title : 71
query_len : 3078
title_len : 277
len_title-query : 266
len_query/title : 1351
q_t_word_match_share : 2963
q_t_jaccard : 605
q_t_common_words : 3027
q_t_total_unique_words : 164
q_t_wc_diff : 130
q_t_wc_ratio : 678
q_t_wc_diff_unique : 707
q_t_wc_ratio_unique : 1931
title_nunique_prefix : 1691
Levenshtein_dis : 119
Levenshtein_ratio : 328
Levenshtein_jaro : 573
Levenshtein_jaro_winkler : 2013
longistStr_len : 2859
prefix_title_pos : 517
prefix_title_relative_pos : 636
len_title-query_word : 225
len_query/title_word : 403
title_query_leven_rate : 699
lcseque_len : 804
preifx_detail_title_pos : 41
cityblock_sim : 908
norm_similarity : 1491
cosine_similarity : 6565
0_lsi : 1838
1_lsi : 540
2_lsi : 1663
3_lsi : 212
4_lsi : 669
5_lsi : 185
6_lsi : 871
7_lsi : 515
8_lsi : 281
9_lsi : 998
10_lsi : 222
11_lsi : 794
12_

# 保存预训练模型

In [4]:
gbm.save_model('/home/kesci/work/first_zzp/model/lgb1200_w2vchange.txt')

# 下面这段代码是加载预训练lgb模型

In [1]:
import lightgbm as lgb
gbm=lgb.Booster(model_file='/home/kesci/work/first_zzp/model/lgb1200.txt')

# 下面是加载测试集

In [17]:
import pandas as pd
test_data=pd.read_pickle("/home/kesci/work/first_zzp/test_all_fea.pickle")
norm(test_data,['title_count','title_sum','title_nunique_prefix'])
test_data

,title_sum,title_count,title_ctr,title_query_dot_similarity,title_query_wmd_dis,query_word_num,title_word_num,is_query_in_title,query_len,title_len,...,10_lsi,11_lsi,12_lsi,13_lsi,14_lsi,15_lsi,16_lsi,17_lsi,18_lsi,19_lsi
0,NaN,NaN,NaN,2.467954,2.962266,5,15,0,21,64,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
1,5.585586e-01,5.590591e-01,0.238788,3.494070,1.819170,5,4,0,21,15,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
2,2.162162e-01,1.826827e-01,0.192738,2.843601,2.426266,5,9,0,21,34,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
3,8.588589e-01,7.862863e-01,0.445409,2.588134,2.903956,5,11,0,21,47,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
4,NaN,NaN,NaN,2.054980,2.537336,5,9,0,21,34,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
5,1.000000e-07,6.806807e-02,0.044410,3.042946,2.488552,5,8,0,21,37,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
6,5.855856e-01,4.819820e-01,0.372021,2.958290,1.730939,5,5,0,21,20,...,0.012029,0.052054,0.024495,-0.059344,-0.061288,-0.004410,-0.040278,-0.003942,-0.026607,-0.046880
7,7.507508e-01,6.861862e-01,0.361941,2.845859,2.596473,4,21,0,17,83,...,0.015682,-0.005775,-0.000173,0.008444,0.005047,-0.001640,-0.010071,0.001998,-0.008235,0.005822
8,8.523524e-01,8.303303e-01,0.326117,2.922247,2.629477,4,18,0,17,77,...,0.015682,-0.005775,-0.000173,0.008444,0.005047,-0.001640,-0.010071,0.001998,-0.008235,0.005822
9,NaN,NaN,NaN,3.120164,2.578096,4,18,0,17,76,...,0.015682,-0.005775,-0.000173,0.008444,0.005047,-0.001640,-0.010071,0.001998,-0.008235,0.005822


In [5]:
result_list=[]
from time import time
for i in range(10):############因为我把最终一亿的测试集分块了，此处是分块读取test集，最后concat
    print(i+1)
    time1=time()
    test_data=pd.read_pickle("/home/kesci/test_final_fea/part"+str(i+1)+".pickle")
    y_pred = gbm.predict(test_data[fea], num_iteration=700)
    y_pred_df=pd.DataFrame(y_pred)
    result_list.append(y_pred_df)
    time2=time()
    print((time2-time1)/60)

1
2.0564005653063457
2
2.1260224620501202
3
2.0961411158243815
4
2.452286008993785
5
2.2341359734535216
6
2.131627678871155
7
2.107110846042633
8
1.8496156533559163
9
1.8544540484746297
10
1.8585427522659301


In [6]:
result_all_test_final=pd.concat(result_list,axis=0)
result_all_test_final

,0
0,0.151121
1,0.149615
2,0.331942
3,0.185651
4,0.313227
5,0.137957
6,0.151461
7,0.352418
8,0.412920
9,0.229125


In [7]:
result_all_test_final.mean()

0    0.196639
dtype: float64

# 下面是不分快

In [18]:
# test_data=test_data[['title_ctr','title_sum','title_count','title_nunique_prefix']]
# fea=list(test_data.columns)
y_pred = gbm.predict(test_data[fea], num_iteration=gbm.best_iteration)
y_pred_df=pd.DataFrame(y_pred)

In [19]:
y_pred_df.mean()

0    0.19933
dtype: float64

# 多进程读取测试集，取其中的query_id和title_id两列，用来做最终的submission

In [22]:
# train_data_ctr=pd.read_csv(data_store_path+"train_data_ctr.csv")#读已经做好ctr特征的训练集
# test_data_ctr=pd.read_csv(data_store_path+"test_data_ctr.csv")#读已经做好ctr特征的测试集
# train_data=pd.read_csv(data_store_path+"train_data.csv")#读训练集（已经用shuffle分好了的）
# train_data=pd.concat([train_data,train_data_ctr],axis=1)
# test_data=pd.concat([test_data,test_data_ctr],axis=1)
# del train_data_ctr
# del test_data_ctr
# train_data
##########################################################################################
# train_data=pd.read_csv(data_store_path+"train_feature_first_no_query_and_title_data.csv")#读已经做好ctr特征的训练集
# test_data=pd.read_csv(data_store_path+"test_feature_first_no_query_and_title_data.csv")#读已经做好ctr特征的测试集
###################################################下面这个是为了去做 相似度特征
# train_data=pd.read_csv(data_store_path+"train_feature_first_data.csv")#读已经做好ctr特征的训练集
# test_data=pd.read_csv(data_store_path+"test_feature_first_data.csv")#读已经做好ctr特征的测试集

############################################
# train_data=pd.read_csv(data_store_path+"data_sets/train_data_ctr.csv")#读已经做好ctr特征的训练集
# test_data=pd.read_csv(data_store_path+"data_sets/test_data_ctr.csv")#读已经做好ctr特征的测试集

# import h5py
# f = h5py.File(data_store_path+"train_data.hd5",'r')   #打开h5文件
# print(f.keys)
# a = f['train']                    #取出主键为data的所有的键值
# f.close()
# print(a)

data_r_path='/home/kesci/input/bytedance/bytedance_contest.final_2.csv'
cpu_num=16
start=0
all_data_num=20000000
names_out=['query_id','query','query_title_id','title']
usecols_out=['query_id','query_title_id']
has_head=False
if has_head==False:
    import multiprocessing
    from time import time
    import pandas as pd
    all_data=[]
    #test_final_part1.csv
    def data_read(start,single_data_num,data_real_path):
        data_out=pd.read_csv(data_real_path,header=None,names=names_out,usecols=usecols_out,skiprows=start,nrows=single_data_num)
        return data_out
    time1=time()
    pool = multiprocessing.Pool(processes=cpu_num)
    
    for epoch in range(int(cpu_num)):
        print(epoch)
        single_data_num=int(all_data_num/cpu_num)
        all_data.append(pool.apply_async(data_read, [start+single_data_num*epoch,single_data_num,data_r_path]))
    # single_data_num=20000000/16
    # all_data_num=20000000
    # for epoch in range(int(all_data_num/single_data_num)):
    #     all_data.append(pool.apply_async(data_read, [single_data_num*epoch,single_data_num,data_path+"test_final_part1.csv"]))
    pool.close()
    pool.join()
    time2=time()
    print(time2-time1)
    all_data_pro=[single.get() for single in all_data]
    time3=time()
    print(time3-time2)
    all_data_pro=pd.concat(all_data_pro)
    test_data_pred_need=all_data_pro.reset_index(drop=True)


# result=pd.concat([test_data_pred_need,y_pred_df],axis=1)
# result.to_csv("first_zzp/result/sub_lgb_1200epoch.csv",header=None,index=None)#####合成三列做 最终的提交结果


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
24.824506759643555
0.0001704692840576172


In [24]:
!wget -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit
!https_proxy="http://klab-external-proxy" ./kesci_submit -file /home/kesci/work/first_zzp/result/sub_lgb_1200epoch.csv -token 02ada54c9760d3e1

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-08-10 16:58:05--  https://www.heywhale.com/kesci_submit
Resolving www.heywhale.com (www.heywhale.com)... 106.15.25.147
Connecting to www.heywhale.com (www.heywhale.com)|106.15.25.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6709558 (6.4M) [application/octet-stream]
Saving to: ‘kesci_submit’

kesci_submit        100%[===================>]   6.40M  21.1MB/s    in 0.3s    

2019-08-10 16:58:05 (21.1 MB/s) - ‘kesci_submit’ saved [6709558/6709558]

Kesci Submit Tool 3.2.1

> 已验证Token
> 提交文件 /home/kesci/work/first_zzp/result/sub_lgb_1200epoch.csv (575321.68 KiB)
> 已上传 100 %
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
